In [3]:
import numpy as np
import random
from tkinter import *
import tkinter.ttk as ttk

In [4]:
class Board:
    def __init__(self,rows=6,columns = 7,goal = 4):
        self.rows = rows
        self.columns = columns
        self.goal = goal
        self.table = np.arange(rows*columns).reshape(rows,columns)*0
        self.fullcolumns = []
    def show(self):
        for i in range(self.rows):
            row = self.table[self.rows-1-i,:]
            row = ['O' if x==1 else x for x in row]
            row = ['X' if x==-1 else x for x in row]
            row = [' ' if x==0 else x for x in row]
            print(*row,sep = " | ")
    def add(self,column,player):
        for i in range(self.rows):
            if (self.table[i,column] == 0):
                self.table[i,column] = player
                if (i == self.rows-1):
                    self.fullcolumns = self.fullcolumns + [column]
                return [i,column]
        return False
    def available(self):
        return [j for j in list(range(self.columns)) if j not in self.fullcolumns]
    def gameover(self):
        #incomplete
        #check rows
        for i in range(self.rows):
            check = 0
            for j in range(self.columns):
                check = (check*self.table[i,j] > 0)*check + self.table[i,j]
                if(abs(check) == self.goal):
                    return [True,np.sign(check)]
        #check columns
        for j in range(self.columns):
            check = 0
            for i in range(self.rows):
                check = (check*self.table[i,j] > 0)*check + self.table[i,j]
                if(abs(check) == self.goal):
                    return [True,np.sign(check)]
        #check diagonals sx
        for k in range(-self.columns+1,self.rows):
            check = 0
            if(k < 0):
                i = 0
                j = -k
            else:
                i = k
                j = 0
            while ((i<self.rows) & (j<self.columns)):
                check = (check*self.table[i,j] > 0)*check + self.table[i,j]
                if(abs(check) == self.goal):
                    return [True,np.sign(check)]
                i = i+1
                j = j+1
        #check diagonals dx
        for k in range(-self.columns+1,self.rows):
            check = 0
            if(k < 0):
                i = 0
                j = -k
            else:
                i = k
                j = self.rows-1
            while ((i<self.rows) & (j<self.columns)):
                check = (check*self.table[i,j] > 0)*check + self.table[i,j]
                if(abs(check) == self.goal):
                    return [True,np.sign(check)]
                i = i+1
                j = j-1
        #full table
        if (sum(sum(self.table == 0)) == 0):
            return [True,0]
        return [False]

In [36]:
class Game:   
    def __init__(self,rows=6,columns = 7,goal = 4):
        self.rows = rows
        self.columns = columns
        self.goal = goal

    def run(self):
        window = Tk()
        window.title("Connect Four")
        window.geometry("700x500")
        window.resizable(0, 0)

        def SetGame():
            newgame.place_forget()
            label1.place(x = 340,y=50, anchor = "center")
            label2.place(x = 340,y=250, anchor = "center")
            button11.place(height = 75, width = 150, x = 150, y = 100)
            button12.place(height = 75, width = 150, x = 375, y = 100)
            button21.place(height = 75, width = 150, x = 150, y = 300)
            button22.place(height = 75, width = 150, x = 375, y = 300)
            button11.configure(foreground="black")
            button12.configure(foreground="black")
            button21.configure(foreground="black")
            button22.configure(foreground="black")
            labelplayer.place_forget()
            labelwinner.place_forget()
            board.place_forget()
            self.player1pc = True
            self.player2pc = True
            self.player1active = False
            self.player2active = False

        def Button11Pressed():
            self.player1active = True
            button11.configure(foreground="green")
            button12.configure(foreground="red")
            if (self.player2active):
                window.after(1000,StartGame)

        def Button12Pressed():
            self.player1active = True
            self.player1pc = False
            button12.configure(foreground="green")
            button11.configure(foreground="red")
            if (self.player2active):
                window.after(1000,StartGame)

        def Button21Pressed():
            self.player2active = True
            button21.configure(foreground="green")
            button22.configure(foreground="red")
            if (self.player1active):
                window.after(1000,StartGame)

        def Button22Pressed():
            self.player2active = True
            self.player1pc = False
            button22.configure(foreground="green")
            button21.configure(foreground="red")
            if (self.player1active):
                window.after(1000,StartGame)

        def StartGame():
            label1.place_forget()
            label2.place_forget()
            button11.place_forget()
            button12.place_forget()
            button21.place_forget()
            button22.place_forget()
            newgame.place(height = 50, width = 100, x = 50, y = 30)
            labelplayer.pack()
            labelplayer.place(x = 610,y=65, anchor = "center")
            board.place(height = 300, width = 600, x = 50, y = 150)
            canvas.place(height = 300, width = 600, x=0,y=0)
            for j in range(self.columns-1):
                canvas.create_line(600*(j+1)/self.columns, 0, 600*(j+1)/self.columns, 300, fill = "black", width = 3)
            for i in range(self.rows-1):
                canvas.create_line(0, 300*(i+1)/self.rows, 600, 300*(i+1)/self.rows, fill = "black", dash = 5, width = .5)
            canvas.create_line(2, 0, 2, 300, fill = "black", width = 3)
            canvas.create_line(597, 0, 597, 300, fill = "black", width = 3)
            canvas.create_line(0, 297, 600, 297, fill = "black", width = 3)
            self.b = Board()
            self.player = 1
            window.after(500,StartLoop)
        
        def StartLoop():
            policy = random.choice(self.b.available())
            self.b.add(policy,self.player)
            self.player = self.player * (-1)
            gameover = self.b.gameover()
            if(gameover[0]):
                if(gameover[1] == 1):
                    labelwinner.configure(text = "Game over.\nO wins.")
                if(gameover[1] == -1):
                    labelwinner.configure(text = "Game over.\nX wins.")
                labelwinner.configure(justify = "center")
                labelwinner.place(x = 350,y=60, anchor = "center")
                return
            if (self.player == 1):
                labelplayer.configure(text = "Player 1: O")
            else:
                labelplayer.configure(text = "Player 2: X")
            window.after(100,StartLoop)

        newgame = Button(window, text = "New game", command = SetGame)
        newgame.place(height = 50, width = 100, x = 50, y = 30)

        label1 = Message(window, text = "Player 1", width = 100)
        label2 = Message(window, text = "Player 2", width = 100)

        button11 = Button(window, text = "PC", command = Button11Pressed)
        button12 = Button(window, text = "Player 1", command = Button12Pressed)
        button21 = Button(window, text = "PC", command = Button21Pressed)
        button22 = Button(window, text = "Player 2", command = Button22Pressed)
        
        labelplayer = Message(window, text = "Player 1: O", width = 100)
        
        board = Frame(window)
        canvas = Canvas(board)

        labelwinner = Message(window,text = "Game over.\nDraw.", width = 100)
        
        window.mainloop()

In [37]:
g = Game()
g.run()